In [7]:
import pandas as pd
from modules import statistic_function as sf
from modules import DataDrop as DD
PATH_GROUP = "./Appolo/A_threeGroupsData/interval_lower_50/Highcc.csv"
PATH_WEATHER = "./raw_data/merged_weatherdata_20220725.csv"
PATH_SAVE = "./Appolo/A_Dataset/highccgroup/PreDataset.csv"

In [8]:
df = pd.read_csv(PATH_GROUP)
df["supplied_at"] = pd.to_datetime(df["supplied_at"], format="%Y-%m-%d %X")
#df.drop(columns=["level_0"], inplace=True)
df_weather = pd.read_csv(PATH_WEATHER)
df_weather = DD.weather_drop(df_weather)
id_unique = pd.unique(df["customer_id"])
sum_mean = sf.Sum_Mean(df, id_unique, "customer_id", "supplied_at", "volume")

df["sum"] = ""
df["mean"] = ""

In [9]:
for id in sum_mean.keys():
    df_id = df[df["customer_id"]==id]
    for year in sum_mean[id].keys():
        for i in df_id.index:
            if df_id.loc[i, "supplied_at"].year == int(year.split("-")[0]) and df_id.loc[i, "supplied_at"].month < 5:
                df.loc[i, "sum"] = sum_mean[id][year][0]
                df.loc[i, "mean"] = sum_mean[id][year][1]
            elif df_id.loc[i, "supplied_at"].year + 1 == int(year.split("-")[0]) and df_id.loc[i, "supplied_at"].month > 10:
                df.loc[i, "sum"] = sum_mean[id][year][0]
                df.loc[i, "mean"] = sum_mean[id][year][1]

In [10]:
consumption_ALL_ID, temperature_ALL_ID, interval_ALL_ID = sf.consumption_temperature(id_unique, df, df_weather,"customer_id", "supplied_at", "volume")
weatherlist = ["matemp", "mltemp", "mhtemp", "mwind", "msnowacc", "msnow", "msun", "mhumidi"]
for i in weatherlist:
    df[i] = ""
# df["mean_temperature"] = ""
df["mean_consumption"] = ""
df["interval"] = ""

for id in id_unique:
    df_id = df[df["customer_id"]==id]
    for key in temperature_ALL_ID[id].keys():
        for i in df_id.index:
            if df_id.loc[i, "supplied_at"] == key:
                for j in range(len(weatherlist)):
                    df.loc[i, weatherlist[j]] = temperature_ALL_ID[id][key][j]
    for key in consumption_ALL_ID[id].keys():
        for i in df_id.index:
            if df_id.loc[i, "supplied_at"] == key:
                df.loc[i, "mean_consumption"] = consumption_ALL_ID[id][key]
    for key in interval_ALL_ID[id].keys():
        for i in df_id.index:
            if df_id.loc[i, "supplied_at"] == key:
                df.loc[i, "interval"] = interval_ALL_ID[id][key]

In [11]:
df.head()

,customer_id,supplied_at,volume,size,sum,mean,matemp,mltemp,mhtemp,mwind,msnowacc,msnow,msun,mhumidi,mean_consumption,interval
0,251.0,2020-11-20 08:26:00,109.8,198,1653.9,11.729787,,,,,,,,,,
1,251.0,2020-12-03 10:20:00,124.3,198,1653.9,11.729787,-2.3,-0.138462,-4.353846,2.730769,1.0,0.769231,1.976923,67.0,9.561538,13
2,251.0,2020-12-18 09:32:00,177.9,198,1653.9,11.729787,0.1,2.486667,-2.766667,2.98,1.333333,0.8,3.8,62.466667,11.86,15
3,251.0,2020-12-29 13:51:00,144.6,198,1653.9,11.729787,2.745455,5.863636,-0.290909,3.036364,11.727273,2.545455,2.390909,68.454545,13.145455,11
4,251.0,2021-01-13 16:15:00,248.7,198,1653.9,11.729787,6.44,10.193333,3.253333,2.34,28.066667,4.266667,3.853333,69.066667,16.58,15


In [12]:
df.to_csv(PATH_SAVE, index=False)